In [1]:
#Visualizacion del dataset.
import pandas as pd

df = pd.read_csv('https://data.nasa.gov/api/views/gh4g-9sfh/rows.csv?accessType=DOWNLOAD')
df.to_csv('../Data/raw/Nasa_MeteoriteLanding.csv')
df

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)"
...,...,...,...,...,...,...,...,...,...,...
45711,Zillah 002,31356,Valid,Eucrite,172.0,Found,1990.0,29.03700,17.01850,"(29.037, 17.0185)"
45712,Zinder,30409,Valid,"Pallasite, ungrouped",46.0,Found,1999.0,13.78333,8.96667,"(13.78333, 8.96667)"
45713,Zlin,30410,Valid,H4,3.3,Found,1939.0,49.25000,17.66667,"(49.25, 17.66667)"
45714,Zubkovsky,31357,Valid,L6,2167.0,Found,2003.0,49.78917,41.50460,"(49.78917, 41.5046)"


In [2]:
#Eliminacion de datos nulos.
df.dropna(inplace=True)
df.isna().sum()

name           0
id             0
nametype       0
recclass       0
mass (g)       0
fall           0
year           0
reclat         0
reclong        0
GeoLocation    0
dtype: int64

In [3]:
# Agregada columna 'mass_group' permitira agrupar meteoritos por masa.
df['mass_group'] = pd.qcut(df['mass (g)'], 10, labels=False)

# Agregadas columnas 'Hemisphere_ns' y 'Hemisphere_ew' que indican en que hemisferio se encontro/cayo cada meteorito. 
df['Hemisphere_ns'] = df['reclat'].apply(lambda x: 'North' if x >= 0 else 'South')
df['Hemisphere_ew'] = df['reclong'].apply(lambda x: 'East' if x >= 0 else 'West')

# Agregada columna 'new_group' que agrupa los meteoritos en base a su masa en 5 tipos (en base a columna mass_group) 
def categorize_mass_group(value):
    if value in [1, 2]:
        return 'muy pequeños'
    elif value in [3, 4]:
        return 'pequeños'
    elif value in [5, 6]:
        return 'medianos'
    elif value in [7, 8]:
        return 'grandes'
    elif value in [9, 10]:
        return 'muy grandes'
    else:
        return value

df['new_group'] = df['mass_group'].apply(categorize_mass_group)

# Agregadas columnas 'geometry' y 'cntry_name' que indican posicion y pais al que corresponde cada coordenada en base a columnas 'reclat' y 'reclong'
import geopandas as gpd
import pandas as pd
import earthpy as et
import shapely
from folium.plugins import MarkerCluster
from shapely.wkt import loads as wkt_loads 
from shapely.geometry import Point

shapefile_path = '../Data/shape/country_shapes.shp'
gdf_countries = gpd.read_file(shapefile_path)
df['geometry'] = df.apply(lambda row: Point(row['reclong'], row['reclat']), axis=1)
gdf_points = gpd.GeoDataFrame(df, geometry='geometry')
gdf_points = gdf_points.set_crs(gdf_countries.crs)
gdf_merged = gpd.sjoin(gdf_points, gdf_countries, how='left', op='within')
df.reset_index(drop=True, inplace=True)
gdf_merged.reset_index(drop=True, inplace=True)

if len(gdf_merged) == len(df):
    df['cntry_name'] = gdf_merged['cntry_name']
else:
    df = df.join(gdf_merged['cntry_name'])

df.head()

C:\Users\ajrob\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,mass_group,Hemisphere_ns,Hemisphere_ew,new_group,geometry,cntry_name
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)",4,North,East,pequeños,POINT (6.08333 50.775),Germany
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)",8,North,East,grandes,POINT (10.23333 56.18333),Denmark
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)",9,North,West,muy grandes,POINT (-113 54.21667),Canada
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)",9,North,West,muy grandes,POINT (-99.9 16.88333),Mexico
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)",8,South,West,grandes,POINT (-64.95 -33.16667),Argentina


In [4]:
#Pasar variables categoricas a numericas
df['new_group_n']=pd.factorize(df['new_group'])[0]                    
df['Hemisphere_ew_n']=pd.factorize(df['Hemisphere_ew'])[0]
df['Hemisphere_ns_n']=pd.factorize(df['Hemisphere_ns'])[0]
df['cntry_name_n']=pd.factorize(df['cntry_name'])[0]
df['GeoLocation_n']=pd.factorize(df['GeoLocation'])[0]
df.head()

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,...,Hemisphere_ns,Hemisphere_ew,new_group,geometry,cntry_name,new_group_n,Hemisphere_ew_n,Hemisphere_ns_n,cntry_name_n,GeoLocation_n
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)",...,North,East,pequeños,POINT (6.08333 50.775),Germany,0,0,0,0,0
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)",...,North,East,grandes,POINT (10.23333 56.18333),Denmark,1,0,0,1,1
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)",...,North,West,muy grandes,POINT (-113 54.21667),Canada,2,1,0,2,2
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)",...,North,West,muy grandes,POINT (-99.9 16.88333),Mexico,2,1,0,3,3
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)",...,South,West,grandes,POINT (-64.95 -33.16667),Argentina,1,1,1,4,4


In [5]:
df.columns

Index(['name', 'id', 'nametype', 'recclass', 'mass (g)', 'fall', 'year',
       'reclat', 'reclong', 'GeoLocation', 'mass_group', 'Hemisphere_ns',
       'Hemisphere_ew', 'new_group', 'geometry', 'cntry_name', 'new_group_n',
       'Hemisphere_ew_n', 'Hemisphere_ns_n', 'cntry_name_n', 'GeoLocation_n'],
      dtype='object')

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

variables = df[['mass (g)', 'year', 'reclat','reclong','new_group_n','Hemisphere_ew_n', 'Hemisphere_ns_n', 'cntry_name_n']]
localizacion = df['GeoLocation_n']

X_train, X_test, y_train, y_test = train_test_split(variables, localizacion, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

modelo_arbol = DecisionTreeClassifier(random_state=42)
modelo_arbol.fit(X_train_scaler, y_train)

DecisionTreeClassifier(random_state=42)

In [8]:
y_pred = modelo_arbol.predict(X_test)

c:\Users\ajrob\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [9]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.16279679916043552


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

modelo_bosque = RandomForestClassifier(n_estimators=10, random_state=42)
modelo_bosque.fit(X_train_scaler, y_train)

MemoryError: could not allocate 1789263872 bytes

In [ ]:
y_pred = modelo_bosque.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

modelo_boosting = GradientBoostingClassifier(random_state=42)
modelo_boosting.fit(X_train_scaler, y_train)

MemoryError: Unable to allocate 3.10 GiB for an array with shape (30492, 13651) and data type float64

In [ ]:
y_pred = modelo_boosting.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

modelo_vecinos = KNeighborsClassifier(n_neighbors=5)
modelo_vecinos.fit(X_train_scaler, y_train)

KNeighborsClassifier()

In [11]:
y_pred = modelo_vecinos.predict(X_test)

c:\Users\ajrob\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


In [12]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.13970877607241244
